# Paper Reproduction Launcher (Llama Runner)

This Colab-friendly notebook drives the existing CLI scripts in `summarization/` to run very small LED and Llama jobs. It uses the `train_last_100.json` / `valid_last_100.json` splits so we can exercise the full code paths quickly before launching the long runs described in the paper.

In [1]:
!sudo apt-get update -y
!sudo apt-get install -y python3.10 python3.10-dev python3.10-distutils

# Point 'python' and 'python3' to python3.10
!sudo update-alternatives --install /usr/bin/python python /usr/bin/python3.10 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1

# Reinstall pip for Python 3.10
!curl -sS https://bootstrap.pypa.io/get-pip.py | sudo python3.10

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://cli.github.com/packages stable InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,901 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555

In [2]:
import sys
sys.version
# assert '3.10' in sys.version
# !python3.10 --version

'3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]'

In [3]:
# @title Sync the local repository from Google Drive (no git clone)
from pathlib import Path
import os
import importlib.util

COLAB = importlib.util.find_spec("google.colab") is not None
if COLAB:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive', force_remount=True)
    REPO_IN_DRIVE = Path('/content/drive/Othercomputers/My Mac/patient_summaries_with_llms/')  # @param {type:"string"}
    if not REPO_IN_DRIVE.exists():
        raise FileNotFoundError(f"Upload/sync the local repo to {REPO_IN_DRIVE} first.")
    TARGET_DIR = Path('/content/patient_summaries_with_llms')
    TARGET_DIR.mkdir(parents=True, exist_ok=True)
    os.system(f"rsync --progress -a --delete '{REPO_IN_DRIVE}/summarization' '{TARGET_DIR}/'")
    os.system(f"rsync --progress -a --delete '{REPO_IN_DRIVE}/data' '{TARGET_DIR}/'")
    os.system(f"rsync --progress -a --delete '{REPO_IN_DRIVE}/requirements.txt' '{TARGET_DIR}/'")
    os.system(f"rsync --progress -a --delete '{REPO_IN_DRIVE}/requirements-llama.txt' '{TARGET_DIR}/'")
    %cd /content/patient_summaries_with_llms
else:
    print("Running outside Colab; using the current working directory.")
    TARGET_DIR = Path.cwd()

Mounted at /content/drive
/content/patient_summaries_with_llms


In [4]:
# @title Install base dependencies shared by LED + evaluation
%pip install -q -r requirements-llama.txt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Llama LoRA smoke run (`summarization/fine_tune_llama.py`)

Runs a tiny LoRA training job on the same dataset subset (100 examples). This assumes you already have access to `meta-llama/Llama-2-7b-hf` and a GPU runtime; set `HF_TOKEN` in the environment if needed.

In [8]:
%pip install bert sacremoses fire bert_score

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 15.2 MB/s  0:00:00
  Created wheel for bert: filename=bert-2.2.0-py3-none-any.whl size=3816 sha256=a9b809737e0d1c831fce341f79f6f690c9dbd0b5f14e07d38f247ed91cef5d41
  Stored in directory: /root/.cache/pip/wheels/d0/97/c8/5775d57c323970511488d9b53ce66cf0ded394fe4b4f6e0afb
  Created wheel for erlastic: filename=erlastic-2.0.0-py3-none-any.whl size=6839 sha256=9955edc98b15b0baf12a5fc497d1276e9cbc5792b9f732ee761a62a1b9010bf5
  Stored in directory: /root/.cache/pip/wheels/f5/ca/b1/ce71c2e0e9e0df3786697d06d53574be03ef86cd4fb0e2cb99
Successfully built bert erlastic
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [fire]


In [10]:
# @title Configure HF token and output directories
import os
from pathlib import Path
from google.colab import userdata


os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
os.environ["WANDB_MODE"] = "offline"
%env MPLBACKEND=Agg

DATA_DIR = Path("data/ann-pt-summ/1.0.1/mimic-iv-note-ext-di-bhc/dataset")
assert DATA_DIR.exists(), f"Missing dataset folder: {DATA_DIR}"

env: MPLBACKEND=Agg


In [13]:
# @title Llama 2 7B LoRA fine-tuning on 100-example subset
!python3.10 summarization/fine_tune_llama.py \
    --model_name_or_path meta-llama/Llama-2-7b-hf \
    --data_path {DATA_DIR} \
    --output_path results/llama_full_run_predict \
    --evaluation \
    --evaluation_model_path data/llama_4000_600_chars/best_val_loss \
    --num_test_examples 100

data: data/ann-pt-summ/1.0.1/mimic-iv-note-ext-di-bhc/dataset
wandb: Tracking run with wandb version 0.16.6
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/01c7f73d771dfac7d292323805ebc428287df4f9/config.json
Model config LlamaConfig {
  "_name_or_path": "meta-llama/Llama-2-7b-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02